In [ ]:
import pandas as pd
# Loading the client to model to generate question decomposition
from google import genai
from tqdm import tqdm
import time
client = genai.Client(api_key="")

In [40]:
df = pd.read_csv("E:\\ASU PhD\\Coursework\\CSE576 Natural Language Processing\\FinalProject\\Code\\results\\results_k10.csv")

In [41]:
all_responses = list()
for ind in tqdm(df.index.values.tolist()):
    gold_answer = df.loc[ind, 'Answer']
    answer = df.loc[ind, 'Generated Answer']

    answer_formatting_prompt = f"""
            You will be given Answer and Gold Answer, you have to Convert the answer into a format of gold answer given above, if the content or meaning is same (semantically same) they should be same.

            Few examples of convertion for your understanding:
            
            answer: ITA, gold answer: Italy. Reasoning- ITA is country code of Italy hence ITA and Italy are same and you can convert ITA to Italy.
                        Your Output: Italy
            answer: 17, gold answer: 17 years. Reasoning- 17 of answer is same as 17 years of the gold answer in the given context of question.
                    Your Output: 17 years
            answer : 10, gold answer: 10. Reasoning- Since, both values are already same no convertion is needed.
                Your Output: 10
            answer : 0, gold answer: 5. Reasoning- Since, both values are semantically not same no convertion is needed for the answer.
                Your Output: 0
            5.⁠ ⁠answer : The answer is not present in the table. , gold answer: 5. Reasoning- Since, both values are semantically not same no convertion is needed for the answer.
                Your Output: The answer is not present in the table.    

            Answer: {answer}
            Gold Answer: {gold_answer}
            Output:"""
    
    response = client.models.generate_content(model='gemini-1.5-pro', contents=answer_formatting_prompt)
    all_responses.append(response)
    

100%|██████████| 42/42 [00:24<00:00,  1.69it/s]


In [54]:
final_responses = [i.text for i in all_responses]
final_responses = [i.split('Output: ')[-1] for i in final_responses]
final_responses = [i.replace('\n', '') for i in final_responses]

In [55]:
df['Edited Answer'] = final_responses

In [56]:
df['TF'] = (df['Answer'] == df['Edited Answer'])

In [57]:
df['TF'].value_counts()

TF
True     31
False    11
Name: count, dtype: int64

In [58]:
display(df[df['TF']==False])

,Unnamed: 0,Question,Answer,Generated Answer,Edited Answer,TF
1,1,A.P. Warrior raced in the Santa Anita Derby at...,blue,"The spires of Churchill Downs, venue of the Ke...","The spires of Churchill Downs, venue of the Ke...",False
5,10,On the logo of the team that played in the Mid...,4,"This image contains no logo, so the question i...","This image contains no logo, so the question i...",False
12,18,The 1980 Frank Converse film poster has a man ...,pink,The woman is wearing orange pants.\n,The woman is wearing orange pants.,False
15,22,On the poster for the TV show in which Tom Mis...,castle,Sleepy Hollow.\n,Sleepy Hollow.,False
16,23,"In the series that Tom Mison appeared in 2009,...",castle,"The misty, rolling landscape of Sleepy Hollow.\n","The misty, rolling landscape of Sleepy Hollow",False
18,25,Regarding the gymnast who had a final all arou...,white,She's standing on the beige part of the mat.\n,She's standing on the beige part of the mat.,False
21,28,Sheila Allen played Ministry WItch#1 in a film...,wand,Goblet\n,Goblet,False
24,31,On the poster for the video game where Billy W...,yes,"No, there is no pig on the poster for *Tiny To...","No, there is no pig on the poster for *Tiny To...",False
25,32,The 1992 film featuring Kirk Baltz has how man...,five,There are six people on the Reservoir Dogs pos...,There are six people on the Reservoir Dogs pos...,False
37,46,Does Mike Amigorena's Toy Story 3 director hav...,no,"Yes, Lee Unkrich, the director of Toy Story 3,...",Yes,False


The woman in the 1990 Frank Converse film poster "Everybody Wins" isn't wearing pants.

